In [ ]:
from tiled.client import from_uri

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
client = from_uri("https://aimm.lbl.gov")
client

In [ ]:
# lets look at some feff data

node = client["xas_feff_nmc"].values_indexer[0]

# print metadata keys
for k in node.metadata:
  print(f"- {k}")
  
# read the datatable
df = node.read()
display(df)

In [ ]:
# we can do the same thing with some experimental data from https://github.com/XraySpectroscopy/XASDataLibrary

node = client["newville"].values_indexer[0]

# print metadata keys
for k in node.metadata:
  print(f"- {k}")
  
# read the datatable
df = node.read()
display(df)

In [ ]:
# we need to register our queries manually since we don't have an aimm-tiled package

from tiled.query_registration import QueryTranslationRegistry, register
from dataclasses import dataclass
import collections
import json

@register(name="raw_mongo")
@dataclass
class RawMongo:
    """
    Run a MongoDB query against a given collection.
    """

    query: str  # We cannot put a dict in a URL, so this a JSON str.

    def __init__(self, query):
        if isinstance(query, collections.abc.Mapping):
            query = json.dumps(query)
        self.query = query
        
@register(name="element")
@dataclass
class ElementQuery:

    symbol: str
    edge: str

    def __init__(self, symbol, edge):
        self.symbol = symbol
        self.edge = edge

In [ ]:
# lets look at all data for the Cobolt K edge

results_feff = client["xas_feff_nmc"].search(ElementQuery(symbol="Co", edge="K"))
print(f"{len(results_feff)=}")

results_newville = client["newville"].search(ElementQuery(symbol="Co", edge="K"))
print(f"{len(results_newville)=}")

In [ ]:
# now we can plot some results

results_feff.values_indexer[0].read().plot("energy", "mu")

df = results_newville.values_indexer[0].read()
df["mu"] = np.log(df["i0"] / df["itrans"])
df.plot("energy", "mu")